# Net

In [1]:

import time
from datetime import datetime
import h5py
#import h5py_cache as h5c
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata
import torch.multiprocessing
import sys
import time
#torch.multiprocessing.set_start_method('spawn')

from hdf5_dataset import H5Dataset




train_data_path = './data/for_train.h5'
test_data_path ='./data/for_test.h5'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Load the test data.
print('Loading test data ...')
test_set = H5Dataset(test_data_path, 0, 10000)
test_loader = torchdata.DataLoader(test_set, batch_size=128, shuffle=True)
data_iter = iter(test_loader)
images, labels = next(data_iter)
num=len(labels)
print('一共有：{}个图片'.format(num))

CHANNELS = [20, 50, 500]
class Net(nn.Module):
    def __init__(self, input_channels):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(CHANNELS[0], CHANNELS[1], 5)
        self.fc1 = nn.Linear(CHANNELS[1] * 12 * 12, CHANNELS[2])
        self.fc2 = nn.Linear(CHANNELS[2], 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net=Net(12)
net.load_state_dict(torch.load('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/final_model/lr0001_64/model.pwf'))
correct = 0
total = 0

print(net)
net.to(device)

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        a=datetime.now()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)#输出预测的类别。1表示行的最大值。
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()    #统计标签与预测相同的个数。
b=datetime.now()

aver_time = (b-a)/num
accuracy = 100.0 * float(correct) / float(total)
print('Accuracy of the network on the test set: %.3f%%' % (
    accuracy))
print('测试时间为：{}'.format(aver_time))


cuda:0
Loading test data ...
Loaded images of shape torch.Size([6894, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([6894, 1]), type torch.int32.
一共有：128个图片
Net(
  (conv1): Conv2d(12, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=7200, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
)
Accuracy of the network on the test set: 66.391%
测试时间为：0:00:00.000063


# ninet

In [1]:

import time
from datetime import datetime
import h5py
#import h5py_cache as h5c
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata
import torch.multiprocessing
import sys
import time
#torch.multiprocessing.set_start_method('spawn')

from hdf5_dataset import H5Dataset

train_data_path = './data/for_train.h5'
test_data_path ='./data/for_test.h5'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Load the test data.
print('Loading test data ...')
test_set = H5Dataset(test_data_path, 0, 10000)
test_loader = torchdata.DataLoader(test_set, batch_size=64, shuffle=True)
data_iter = iter(test_loader)
images, labels = next(data_iter)
num=len(labels)
print('一共有：{}个图片'.format(num))

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
                                                          X, self.gamma, self.beta, self.moving_mean,
                                                          self.moving_var, eps=1e-5, momentum=0.9)
        return Y


def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        BatchNorm(out_channels, num_dims=4),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk


import torch.nn.functional as F


class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])


class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()

    def forward(self, x):  # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

net = nn.Sequential(
    nin_block(12, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 2, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    FlattenLayer())

net=net

net.load_state_dict(torch.load('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/final_model/lr0001_ninnet_bn_200epoch_00005lr/model_65.12909776617349.pwf'))
correct = 0
total = 0

print(net)
net.to(device)

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        a=datetime.now()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)#输出预测的类别。1表示行的最大值。
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()    #统计标签与预测相同的个数。
b=datetime.now()

aver_time = (b-a)/num
accuracy = 100.0 * float(correct) / float(total)
print('Accuracy of the network on the test set: %.3f%%' % (
    accuracy))
print('测试时间为：{}'.format(aver_time))

cuda:0
Loading test data ...
Loaded images of shape torch.Size([6894, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([6894, 1]), type torch.int32.
一共有：64个图片
Sequential(
  (0): Sequential(
    (0): Conv2d(12, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): BatchNorm()
    (2): ReLU()
    (3): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
    (6): ReLU()
  )
  (1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm()
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (6): ReLU()
  )
  (3): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1

# one_fl

In [2]:

import time
from datetime import datetime
import h5py
#import h5py_cache as h5c
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata
import torch.multiprocessing
import sys
import time
#torch.multiprocessing.set_start_method('spawn')

from hdf5_dataset import H5Dataset

train_data_path = './data/for_train.h5'
test_data_path ='./data/for_test.h5'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Load the test data.
print('Loading test data ...')
test_set = H5Dataset(test_data_path, 0, 10000)
test_loader = torchdata.DataLoader(test_set, batch_size=128, shuffle=True)
data_iter = iter(test_loader)
images, labels = next(data_iter)
num=len(labels)
print('一共有：{}个图片'.format(num))

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
                                                          X, self.gamma, self.beta, self.moving_mean,
                                                          self.moving_var, eps=1e-5, momentum=0.9)
        return Y
class one_fl_Net_con_nomal(nn.Module):
    def __init__(self, input_channels):
        super(one_fl_Net_con_nomal, self).__init__()

        self.a_1 = nn.Conv2d(12, 96, kernel_size=11, stride=4, padding=0)
        self.batch_1 = BatchNorm(96, num_dims=4)
        self.a_one = nn.Conv2d(96, 96, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.b_1 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.batch_2 = BatchNorm(256, num_dims=4)
        self.b_one = nn.Conv2d(256, 256, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.c_1 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.batch_3 = BatchNorm(384, num_dims=4)
        self.c_one = nn.Conv2d(384, 384, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.consoft = nn.Conv2d(384, 50, kernel_size=1, stride=1, padding=1)
        self.fc1 = nn.Linear(50 * 12 * 12, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.batch_1(self.a_1(x)))
        x = F.relu(self.a_one(F.relu(self.a_one(x))))
        x = self.pool(x)

        x = F.relu(self.batch_2(self.b_1(x)))
        x = F.relu(self.b_one(F.relu(self.b_one(x))))
        x = self.pool(x)

        x = F.relu(self.batch_3(self.c_1(x)))
        x = F.relu(self.c_one(F.relu(self.c_one(x))))
        x = self.pool(x)

        x = self.consoft(x)

        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



net = one_fl_Net_con_nomal(12)

net.load_state_dict(torch.load('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/final_model/lr00005_64_one_fl_conv_batch/model_70.01740644038294.pwf'))
correct = 0
total = 0

print(net)
net.to(device)

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        a=datetime.now()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)#输出预测的类别。1表示行的最大值。
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()    #统计标签与预测相同的个数。
b=datetime.now()

aver_time = (b-a)/num
accuracy = 100.0 * float(correct) / float(total)
print('Accuracy of the network on the test set: %.3f%%' % (
    accuracy))
print('测试时间为：{}'.format(aver_time))

cuda:0
Loading test data ...
Loaded images of shape torch.Size([6894, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([6894, 1]), type torch.int32.
一共有：128个图片
one_fl_Net_con_nomal(
  (a_1): Conv2d(12, 96, kernel_size=(11, 11), stride=(4, 4))
  (batch_1): BatchNorm()
  (a_one): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (b_1): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_2): BatchNorm()
  (b_one): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (c_1): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_3): BatchNorm()
  (c_one): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1))
  (consoft): Conv2d(384, 50, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=7200, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)
Accuracy of the network on the test set:

# fc2nin

In [2]:
import time
from datetime import datetime
import h5py
#import h5py_cache as h5c
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata
import torch.multiprocessing
import sys
import time
#torch.multiprocessing.set_start_method('spawn')

from hdf5_dataset import H5Dataset

train_data_path = './data/for_train.h5'
test_data_path ='./data/for_test.h5'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Load the test data.
print('Loading test data ...')
test_set = H5Dataset(test_data_path, 0, 10000)
test_loader = torchdata.DataLoader(test_set, batch_size=64, shuffle=True)
data_iter = iter(test_loader)
images, labels = next(data_iter)
num=len(labels)
print('一共有：{}个图片'.format(num))

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y
##############           net
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        BatchNorm(out_channels, num_dims=4),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk


import torch.nn.functional as F


class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])


class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()

    def forward(self, x):  # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)


net = nn.Sequential(
    #     nin_block(12, 96, kernel_size=11, stride=4, padding=0),
    #     nn.MaxPool2d(kernel_size=2, stride=1),
    #     nin_block(96, 256, kernel_size=5, stride=1, padding=2),

    #     nn.MaxPool2d(kernel_size=2, stride=1),
    nn.Conv2d(12, 20, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nin_block(50, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 2, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    FlattenLayer())



net = net

net.load_state_dict(torch.load('/home/wuxr/graspping/gpd-master/pytorch/gpd_orgin/final_model/lr00005_fc2nin_bn_200epoch_3/model_68.13170873223092.pwf'))
correct = 0
total = 0

print(net)
net.to(device)

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        a=datetime.now()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)#输出预测的类别。1表示行的最大值。
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()    #统计标签与预测相同的个数。
b=datetime.now()

aver_time = (b-a)/num
accuracy = 100.0 * float(correct) / float(total)
print('Accuracy of the network on the test set: %.3f%%' % (
    accuracy))
print('测试时间为：{}'.format(aver_time))

cuda:0
Loading test data ...
Loaded images of shape torch.Size([6894, 60, 60, 12]), type torch.uint8, and labels of shape torch.Size([6894, 1]), type torch.int32.
一共有：64个图片
Sequential(
  (0): Conv2d(12, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Sequential(
    (0): Conv2d(50, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm()
    (2): ReLU()
    (3): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1))
    (6): ReLU()
  )
  (7): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (8): Dropout(p=0.5, inplace=False)
  (9): Sequential(
    (0): Conv2d(384, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1